## We want to build a model of demand and supply of publicly provided services in a city

In [86]:
import geopy
import numpy as np
from sklearn import gaussian_process
from matplotlib import pyplot as plt 

In [12]:
gaussKern = gaussian_process.kernels.RBF

In [108]:
from enum import Enum

class AgeGroups(Enum):
    Newborn= (0, 3)
    Child= (4,15)
    Young= (16,25)
    Junior= (26,35)
    Senior= (36, 50)
    Over50= (50, 64)
    Over65= (66, 80)
    Over80= (81, 200)
    def __init__(self, startAge, endAge):
        self.start = startAge
        self.end = endAge
    @staticmethod
    def all():
        return([g for g in AgeGroups])
    @property
    def range(self): return self.end - self.start
    
    
class Services(Enum):
    School = 1
    Football = 2
    Doctor = 3
    #etc
    
kPropagationTest = 3

class ServiceUnit:
    def __init__(self, name='', position=(45.4641, 9.1919), users=AgeGroups.all(), times=[], scale=1):
        assert isinstance(position, tuple) & (len(position) == 2), 'Position must be a pair tuple' #will be replaced by nicer class
        self.name = name
        self.site = position  # a Service can have many sites, and a site is not uniquely assigned to a service
        self.times = times
        
        # how the service availablity area varies for different age groups
        self.propagation ={g: (0.04 + .001*np.round(np.random.normal(),2))*kPropagationTest*scale for g in users} 
        self.kernel = {g: gaussKern(length_scale=l) for g, l in self.propagation.items()}
    def evaluate(self, targetPosition, ageGroup):
        assert targetPosition.shape[1] == 2, '2D targets expected'
        reshapedPos = np.array(self.site).reshape(-1,2)
        score = self.kernel[ageGroup](targetPosition, reshapedPos)
        return score
    
    @property
    def users(self): return list(self.propagation.keys())
        

In [109]:
test = ServiceUnit('Duomo')
x = np.random.uniform(low=[45.40,9.1],high=[45.50, 9.3], size=(50,2))
test.evaluate(x, AgeGroups.Child)


array([[0.94358796],
       [0.94583203],
       [0.93679249],
       [0.79850767],
       [0.81640382],
       [0.94534615],
       [0.66261939],
       [0.92596086],
       [0.92846878],
       [0.86843039],
       [0.81263582],
       [0.72698795],
       [0.73466976],
       [0.98459803],
       [0.92903625],
       [0.89356245],
       [0.97061244],
       [0.83645162],
       [0.70735529],
       [0.91337616],
       [0.95156427],
       [0.74791692],
       [0.89838778],
       [0.82171177],
       [0.87821782],
       [0.69510683],
       [0.95652683],
       [0.78632085],
       [0.9061027 ],
       [0.95888333],
       [0.95191051],
       [0.89023853],
       [0.90504684],
       [0.89590066],
       [0.89887565],
       [0.84305503],
       [0.78322774],
       [0.75246128],
       [0.77653393],
       [0.73313224],
       [0.90814996],
       [0.73337167],
       [0.96761395],
       [0.78573449],
       [0.96339957],
       [0.92655408],
       [0.90321286],
       [0.960

In [103]:
np.array((3,4))

array([3, 4])

In [4]:
### Supply modeling
